### Installations and Setup

In [1]:
import numpy as np
import pandas as pd

In [29]:
import random

In [7]:
# emissions are in tCO2e, sales volume is in millions, debt + equity is millions USD, all from 2021 annual reports
# may need to turn into dict later, see if streamlit needs it
# volvo is volva cards only
data = pd.read_csv('auto-client-data.csv', header = 0)

In [8]:
data.head(10)

,company,scope1,scope2,scope3,sales_vol,debt_eq
0,Tesla,185000,403000,1954000,0.94,62131
1,Stellantis,1641028,2233459,523300000,6.50,87910
2,Subaru,232070,359408,28026000,0.86,14805
3,Kia,365100,774000,67563304,2.66,52863
4,Nissan,697851,1541276,127735901,3.88,149862
5,Hyundai,723966,1660238,101790793,3.89,204446
6,Honda,1120000,3380000,249980000,4.55,186383
7,BMW,699713,138849,121705368,2.62,76533
8,Volvo,77000,45000,19561000,0.70,33094


### Getting user selection

In [43]:
# assume user input must add up to 500 mill, and if its worth it, enforce increments of 50 mill  
# must select 4 
selection = {'Tesla': 150, 'Honda': 50, 'Volvo': 200, 'Kia': 100}
funds = 500
if sum(selection.values()) == funds:
    print("Fully allocated")
else:
    print("Not fully allocated") # not enforced yet

if len(selection) == 4:
    print("4 Selected")
else:
    print("Please select 4") # not enforced yet

# plus 50 increment enforcing if can't do through streamlit

Fully allocated
4 Selected


### Operations based on user selection

In [22]:
# Take slice of data based on user selection
df = data[data.company.isin(selection.keys())]

In [23]:
df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq
0,Tesla,185000,403000,1954000,0.94,62131
3,Kia,365100,774000,67563304,2.66,52863
6,Honda,1120000,3380000,249980000,4.55,186383
8,Volvo,77000,45000,19561000,0.70,33094


In [ ]:
# create column for authorized amount
df.loc[:,'authorized'] = df['company'].map(selection)

df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq,authorized
0,Tesla,185000,403000,1954000,0.94,62131,150
3,Kia,365100,774000,67563304,2.66,52863,100
6,Honda,1120000,3380000,249980000,4.55,186383,50
8,Volvo,77000,45000,19561000,0.70,33094,200


In [35]:
# create column for amount outstanding (random value upwards of 50%)
df.loc[:,'outstanding'] = df.loc[:,'authorized'].apply(lambda x: x* random.choice([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]))

In [36]:
df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq,authorized,outstanding
0,Tesla,185000,403000,1954000,0.94,62131,150,135.0
3,Kia,365100,774000,67563304,2.66,52863,100,90.0
6,Honda,1120000,3380000,249980000,4.55,186383,50,45.0
8,Volvo,77000,45000,19561000,0.70,33094,200,180.0


In [41]:
# calculate per client total emissions in megatonnes
df.loc[:,'total_em'] = round((df.loc[:,'scope1'] + df.loc[:,'scope2'] + df.loc[:,'scope3'])/1000000, 4)
# calculate per client emissions intensity proxy
df.loc[:,'prod_intensity'] = round(df.loc[:,'total_em'] / df.loc[:,'sales_vol'], 2) # tonnes in millions and volume is also in millions

In [42]:
df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq,authorized,outstanding,total_em,prod_intensity
0,Tesla,185000,403000,1954000,0.94,62131,150,135.0,2.5420,2.70
3,Kia,365100,774000,67563304,2.66,52863,100,90.0,68.7024,25.83
6,Honda,1120000,3380000,249980000,4.55,186383,50,45.0,254.4800,55.93
8,Volvo,77000,45000,19561000,0.70,33094,200,180.0,19.6830,28.12


In [48]:
# calculcate attribution ratio
df.loc[:,'attr_ratio'] = df.loc[:,'outstanding'] / df.loc[:,'debt_eq']

# calculate portfolio percentage
df.loc[:,'perc'] = df.loc[:,'authorized'] / funds

In [49]:
df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq,authorized,outstanding,total_em,prod_intensity,attr_ratio,perc
0,Tesla,185000,403000,1954000,0.94,62131,150,135.0,2.5420,2.70,0.002173,0.3
3,Kia,365100,774000,67563304,2.66,52863,100,90.0,68.7024,25.83,0.001703,0.2
6,Honda,1120000,3380000,249980000,4.55,186383,50,45.0,254.4800,55.93,0.000241,0.1
8,Volvo,77000,45000,19561000,0.70,33094,200,180.0,19.6830,28.12,0.005439,0.4


In [ ]:
# calculate absolute financed emissions in megattones (maybe when printing out results can express in tonnes?)
df.loc[:,'abs_em'] = df.loc[:,'attr_ratio'] * df.loc[:,'total_em']
df.loc[:,'abs_em_tonnes'] = round((df['attr_ratio'] * df['total_em'])*1000000,0) # add a type cast instead of the rounding to avoid the .0


In [ ]:
# scrap code
# print(f'Out of the ${row['authorized']} million loaned to {row['company']}, ${row['oustanding']} million is outstanding')

In [ ]:
for index, row in df.iterrows():
    print(f'The ${row['authorized']} million loaned to {row['company']} generated {row['abs_em_tonnes']} tCO2e')

The $150 million loaned to Tesla generated 5523.0 tCO2e
The $100 million loaned to Kia generated 116967.0 tCO2e
The $50 million loaned to Honda generated 61441.0 tCO2e
The $200 million loaned to Volvo generated 107057.0 tCO2e


In [ ]:
# gotttta be a better way to output this
print("Let's look at an example to understand how this was calculated")
print(f"You loaned ${df.loc[0,'authorized']} million to {df.loc[0,'company']} ")
print(f"Of that ${df.loc[0,'authorized']} million, {df.loc[0,'company']} used ${df.loc[0,'outstanding']} million")
print(f"This is used to calculate your attribution ratio, representing your contribution to {df.loc[0,'company']}'s activities for the year")
print
# look for better explanation of attributuion ratio

Let's look at an example to understand how this was calculated
You loaned $150 million to Tesla 
Of that $150 million, Tesla used 135.0 
This is used to calculate your attribution ratio, representing your contribution to  Tesla's activities for the year


In [ ]:
print(f"""
Let's look at an example to understand how these were calculated
You loaned ${df.loc[0,'authorized']} million to {df.loc[0,'company']}
Of that ${df.loc[0,'authorized']} million, {df.loc[0,'company']} used ${df.loc[0,'outstanding']} million
This is used to calculate your attribution ratio, representing your contribution to {df.loc[0,'company']}'s activities for the year

""")


Let's look at an example to understand how this was calculated
You loaned $150 million to Tesla
Of that $150 million, Tesla used $135.0 million
This is used to calculate your attribution ratio, representing your contribution to Tesla's activities for the year




In [68]:
portfolio_intensity = np.average(df['prod_intensity'], weights=df['perc'])

print(f'The overall emissions intensity (normalized by vehicles sold in the same period) of this portfolio is {portfolio_intensity} ')

The overall emissions intensity (normalized by vehicles sold in the same period) of this portfolio is 22.817 


In [61]:
df.head()

,company,scope1,scope2,scope3,sales_vol,debt_eq,authorized,outstanding,total_em,prod_intensity,attr_ratio,perc,abs_em,abs_em_tonnes
0,Tesla,185000,403000,1954000,0.94,62131,150,135.0,2.5420,2.70,0.002173,0.3,0.005523,5523.0
3,Kia,365100,774000,67563304,2.66,52863,100,90.0,68.7024,25.83,0.001703,0.2,0.116967,116967.0
6,Honda,1120000,3380000,249980000,4.55,186383,50,45.0,254.4800,55.93,0.000241,0.1,0.061441,61441.0
8,Volvo,77000,45000,19561000,0.70,33094,200,180.0,19.6830,28.12,0.005439,0.4,0.107057,107057.0


In [ ]:
# calculate physical emissions intensity for portfolio
